## 눈깜빡이 게임

In [1]:
totalTime = 30  # 총 게임 시간은 20초로 지정
userID = '000000001'

import os
import random
import cv2
from cvzone.FaceMeshModule import FaceMeshDetector
import cvzone
import numpy as np
from PIL import ImageFont, ImageDraw, Image
import eyeTest as et
import mediapipe as mp
import pandas as pd
import time
import datetime

eye_dist = 33

now = datetime.datetime.now()
nowDatetime = now.strftime('%Y-%m-%d %H:%M:%S')

font = ImageFont.truetype('../static/fonts/H2GSRB.TTF', 35)
small_font = ImageFont.truetype('../static/fonts/NanumGothicBold.otf', 10)
logo = cv2.resize(cv2.imread('../static/button/eye.png', cv2.IMREAD_UNCHANGED),(80,80))

cap = cv2.VideoCapture(0)
cap.set(3, 1280)
cap.set(4, 720)

detector = FaceMeshDetector(maxFaces=1)
mp_hands = mp.solutions.hands

detector = FaceMeshDetector(maxFaces=1)
idList = [0, 17, 78, 292]

# import images
folderBirds = '../static/image/game/'
ListBirds = os.listdir(folderBirds)
catch = []
for object in ListBirds:
    catch.append(cv2.resize(cv2.imread(f'{folderBirds}/{object}', cv2.IMREAD_UNCHANGED),(80,80)))
background = cv2.resize(cv2.imread('../static/image/background.jpg'),(1000,630))

currentObject = catch[0]
pos = [300, 128]
speed = random.randint(5, 10)
count = 0
global birds
birds = True
gameOver = False

ratioList = []
blinkCounter = 0
counter = 0
List = []
finalList = []
selectionSpeed = 8
btn_size = 40
timeStart = time.time()  # 시작 시간은 웹캠이 열리는 시점의 현재시간

def text_def(xy,text_name,fontstyle,text_color):
    global image
    image = Image.fromarray(image)    
    draw = ImageDraw.Draw(image)    
    draw.text(xy=xy,  text=text_name, font=fontstyle, fill= text_color)
    image = np.array(image) 

def resetObject():
    global birds
    pos[0] = random.randint(0, 1180)
    pos[1] = 0
    currentObject = catch[random.randint(0, 3)]
    birds = True
    return currentObject

with mp_hands.Hands(max_num_hands=2,min_detection_confidence=0.5,min_tracking_confidence=0.5) as hands:
    timeEnd = time.time() 
    while True:
        success, image = cap.read()
        image = cv2.flip(image, 1)
        results = hands.process(image)
        et.overlay(image, *(50,50), 40, 40, logo) 
        if time.time()-timeStart < totalTime:      
            if gameOver is False:
                text_def((100, 35),  '눈을 깜빡하면서 새를 잡아보세요.', font, (0, 0, 0))
                image, faces = detector.findFaceMesh(image, draw=False)
                image = cvzone.overlayPNG(image, currentObject, pos)
                pos[1] += speed

                if pos[1] > 520:
                    currentObject = resetObject()

                if faces:
                    face = faces[0]
                    up = face[159]  # Lefteye
                    down = face[23]
                    left = face[130]
                    right = face[243]

                    upDown, _ = detector.findDistance(up, down)
                    leftRight, _ = detector.findDistance(left, right)

                    cx, cy = (up[0] + down[0]) // 2, (up[1] + down[1]) // 2
                    dist, _ = detector.findDistance((cx, cy), (pos[0] + 50, pos[1] + 50))

                    ratio = int((upDown / leftRight) * 100)
                    ratioList.append(ratio)
                    if len(ratioList) > 3:
                        ratioList.pop(0)
                    ratioAvg = sum(ratioList) / len(ratioList)
                    if ratioAvg < eye_dist and counter == 0:
                        eyeStatus = "Open"
                        blinkCounter += 1
                        color = (0,200,0)
                        counter = 1
                    if counter != 0:
                        eyeStatus = "Closed"
                        counter += 1
                        if counter > 10:
                            counter = 0
                            color = (255,0, 255)
                            if birds:
                                currentObject = resetObject()
                                count += 1
                                List.append({
                                    'ID':userID,
                                    '시간':nowDatetime,
                                    '점수':str(count)
                                    })
                            else:
                                gameOver = True
                        cv2.putText(image, eyeStatus, (1050, 680), cv2.FONT_HERSHEY_COMPLEX, 2, (255, 0, 255), 2)
                    else:      
                        List.append({
                            'ID':userID,
                            '시간':nowDatetime,
                            '점수':str(count)
                            })  
                    cvzone.putTextRect(image, f'Score:  {str(count)}',(50,680),scale=2, colorR=(255, 0, 255))
            cv2.putText(image, f'Time: {int(totalTime-(time.time()-timeStart))}', (1000, 80), 1,3,(255, 0, 255),3)
        else: 
            et.img_size(image, background, 630, 360)    

            cv2.putText(image, "Game Over", (320, 300), cv2.FONT_HERSHEY_PLAIN, 7, (255, 0, 255), 10)
            cv2.putText(image, f'Your Score: {str(count)}', (450, 400), 1,3, (255, 0, 255),4)
            
            text_def((460, 450),f'{int(6+totalTime-(time.time()-timeStart))} 초후 종료합니다.',ImageFont.truetype('../static/fonts/H2GSRB.TTF', 40),(0, 0, 0))
            if int(6+totalTime-(time.time()-timeStart)) == 0: 
                break
            
        cv2.imshow("Image", image)
        key = cv2.waitKey(1)
        if key == ord('q'):
            break
        
final_List = List[-1]
finalList.append(final_List)
df = pd.DataFrame(finalList)
df.to_csv('../static/csv파일/게임.csv')      
print(finalList)
print('저장 완료됬습니다.')

cap.release()
cv2.destroyAllWindows()

[{'ID': '000000001', '시간': '2023-03-01 16:03:47', '점수': '32'}]
저장 완료됬습니다.
